In [124]:
import pandas as pd
from elasticsearch import Elasticsearch, helpers
df = pd.read_csv('products-all-data.csv')
df = df[['products_id', 'products_name', 'category', 'rating', 'rating_count', 'review_count']]
df['products_id'] = df['products_id'].astype(str)
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme':'http'}])
def index_products(df):
    actions = [
        {
            "_index": "products",
            "_id": row['products_id'],
            "_source": {
                "products_id": row['products_id'],
                "products_name": row['products_name'],
                "category": row['category'],
                "rating": row['rating'],
                "rating_count": row['rating_count'],
                "review_count": row['review_count']
            }
        }
        for _, row in df.iterrows()
    ]
    helpers.bulk(es, actions)
index_products(df)

C:\Users\Sayantan\AppData\Local\Temp\ipykernel_12852\2282408070.py:5: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('products-all-data.csv')
C:\Users\Sayantan\AppData\Local\Temp\ipykernel_12852\2282408070.py:33: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  helpers.bulk(es, actions)


In [125]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
from tensorflow.keras.regularizers import l2

class ProductModel(tfrs.Model):
    def __init__(self, df):
        super().__init__()
        embedding_dimension = 32
        self.product_model = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=df['products_id'].unique(), mask_token=None),
            tf.keras.layers.Embedding(len(df['products_id'].unique()) + 1, embedding_dimension)
        ])
        self.rating_model = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation="relu", kernel_regularizer=l2(0.01)),
            tf.keras.layers.Dense(64, activation="relu", kernel_regularizer=l2(0.01)),
            tf.keras.layers.Dense(1)
        ])
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )
    def compute_loss(self, features, training=False):
        product_embeddings = self.product_model(features["products_id"])
        ratings = features["rating"]
        rating_predictions = self.rating_model(
            tf.concat([product_embeddings], axis=1))
        return self.task(
            labels=ratings,
            predictions=rating_predictions,
        )

    def predict_scores(self, product_ids):
        product_ids = [str(pid) for pid in product_ids]
        product_embeddings = self.product_model(tf.constant(product_ids))
        scores = self.rating_model(product_embeddings)
        return scores

In [126]:
dataset = tf.data.Dataset.from_tensor_slices(dict(df))
dataset = dataset.shuffle(10000).batch(256)
model = ProductModel(df)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.01))
model.fit(dataset, epochs=50)

def get_similar_products(product_name, num_recommendations=5):
    search_body = {
        "query": {
            "match": {
                "products_name": product_name
            }
        },
        "size": 50
    }
    res = es.search(index="products", body=search_body)
    if res['hits']['total']['value'] == 0:
        return []
    candidates = []
    for hit in res['hits']['hits']:
        candidates.append({
            "products_id": hit['_source']['products_id'],
            "products_name": hit['_source']['products_name'],
            "category": hit['_source']['category'],
            "rating": hit['_source']['rating'],
            "rating_count": hit['_source']['rating_count'],
            "review_count": hit['_source']['review_count']
        })
    candidate_ids = [candidate["products_id"] for candidate in candidates]
    candidate_scores = model.predict_scores(candidate_ids)
    for i, score in enumerate(candidate_scores):
        candidates[i]["score"] = score.numpy()[0]
    sorted_candidates = sorted(candidates, key=lambda x: x["score"], reverse=True)
    return sorted_candidates[:num_recommendations]

product_name = 'attractive table ringer'
recommendations = get_similar_products(product_name, num_recommendations=5)
print("Recommendations:")
for rec in recommendations:
    print(f"Product ID: {rec['products_id']}, Product Name: {rec['products_name']}, Score: {rec['score']}, Category: {rec['category']}, Rating: {rec['rating']}, Rating Count: {rec['rating_count']}, Review Count: {rec['review_count']}")

Epoch 1/50
111/111 [==============================] - 2s 4ms/step - root_mean_squared_error: 1.8891 - loss: 3.5795 - regularization_loss: 1.4866 - total_loss: 5.0660
Epoch 2/50
111/111 [==============================] - 0s 3ms/step - root_mean_squared_error: 1.8537 - loss: 3.4138 - regularization_loss: 1.2936 - total_loss: 4.7073
Epoch 3/50
111/111 [==============================] - 0s 3ms/step - root_mean_squared_error: 1.8585 - loss: 3.4495 - regularization_loss: 1.1259 - total_loss: 4.5754
Epoch 4/50
111/111 [==============================] - 0s 3ms/step - root_mean_squared_error: 1.8565 - loss: 3.4146 - regularization_loss: 0.9808 - total_loss: 4.3954
Epoch 5/50
111/111 [==============================] - 0s 3ms/step - root_mean_squared_error: 1.8606 - loss: 3.4575 - regularization_loss: 0.8548 - total_loss: 4.3123
Epoch 6/50
111/111 [==============================] - 0s 3ms/step - root_mean_squared_error: 1.8604 - loss: 3.4383 - regularization_loss: 0.7457 - total_loss: 4.1840
Epoc

C:\Users\Sayantan\AppData\Local\Temp\ipykernel_12852\1958149064.py:21: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index="products", body=search_body)


In [10]:
import pandas as pd
from elasticsearch import Elasticsearch, helpers
import tensorflow as tf
import tensorflow_recommenders as tfrs
from tensorflow.keras.regularizers import l2

# Load the CSV file
df = pd.read_csv('products-all-data.csv')
df = df[['products_id', 'products_name', 'category', 'base_sub_category', 'products_mrp', 'rating', 'rating_count', 'review_count']]
df['products_id'] = df['products_id'].astype(str)

# Define the ProductModel class
class ProductModel(tfrs.Model):
    def __init__(self, df):
        super().__init__()
        embedding_dimension = 32
        self.product_model = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=df['products_id'].unique(), mask_token=None),
            tf.keras.layers.Embedding(len(df['products_id'].unique()) + 1, embedding_dimension)
        ])
        self.rating_model = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation="relu", kernel_regularizer=l2(0.01)),
            tf.keras.layers.Dense(64, activation="relu", kernel_regularizer=l2(0.01)),
            tf.keras.layers.Dense(1)
        ])
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def call(self, features):
        product_embeddings = self.product_model(features["products_id"])
        rating_predictions = self.rating_model(tf.concat([product_embeddings], axis=1))
        return rating_predictions

    def compute_loss(self, features, training=False):
        labels = features["rating"]
        predictions = self(features)
        return self.task(labels=labels, predictions=predictions)

    def get_product_embeddings(self, product_ids):
        product_ids = [str(pid) for pid in product_ids]
        product_embeddings = self.product_model(tf.constant(product_ids))
        return product_embeddings

# Convert dataframe to TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices(dict(df))
dataset = dataset.shuffle(10000).batch(256)

# Train and save the model
model = ProductModel(df)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.01))
model.fit(dataset, epochs=50)
model.save('product_recommender_model')

# Initialize Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

# Index products into Elasticsearch
def index_products(df):
    actions = [
        {
            "_index": "products",
            "_id": row['products_id'],
            "_source": {
                "products_id": row['products_id'],
                "products_name": row['products_name'],
                "category": row['category'],
                "base_sub_category": row['base_sub_category'],
                "products_mrp": row['products_mrp']
            }
        }
        for _, row in df.iterrows()
    ]
    helpers.bulk(es, actions)

# Uncomment the next line if you need to re-index the data
# index_products(df)

# Define the function to get similar products
def get_similar_products(product_name, num_recommendations=5):
    search_body = {
        "query": {
            "match": {
                "products_name": product_name
            }
        },
        "size": 50
    }
    res = es.search(index="products", body=search_body)
    if res['hits']['total']['value'] == 0:
        return []
    candidates = []
    for hit in res['hits']['hits']:
        candidates.append({
            "products_id": hit['_source']['products_id'],
            "products_name": hit['_source']['products_name'],
            "category": hit['_source']['category'],
            "base_sub_category": hit['_source']['base_sub_category'],
            "products_mrp": hit['_source']['products_mrp']
        })
    candidate_ids = [candidate["products_id"] for candidate in candidates]
    product_embeddings = model.get_product_embeddings(candidate_ids)
    scores = model.rating_model(product_embeddings)
    for i, score in enumerate(scores):
        candidates[i]["score"] = score.numpy()[0]
    sorted_candidates = sorted(candidates, key=lambda x: x["score"], reverse=True)
    return sorted_candidates[:num_recommendations]

# Example usage
product_name = 'attractive table ringer'
recommendations = get_similar_products(product_name, num_recommendations=5)
print("Recommendations:")
for rec in recommendations:
    print(f"Product Name: {rec['products_name']}, Category: {rec['category']}, Subcategory: {rec['base_sub_category']}, Price: {rec['products_mrp']}")


C:\Users\Sayantan\AppData\Local\Temp\ipykernel_9564\3935803115.py:8: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('products-all-data.csv')


Epoch 1/50
111/111 [==============================] - 1s 3ms/step - root_mean_squared_error: 1.9024 - loss: 3.6086 - regularization_loss: 1.4924 - total_loss: 5.1011
Epoch 2/50
111/111 [==============================] - 0s 3ms/step - root_mean_squared_error: 1.8570 - loss: 3.4288 - regularization_loss: 1.2991 - total_loss: 4.7279
Epoch 3/50
111/111 [==============================] - 0s 3ms/step - root_mean_squared_error: 1.8579 - loss: 3.4318 - regularization_loss: 1.1309 - total_loss: 4.5626
Epoch 4/50
111/111 [==============================] - 0s 3ms/step - root_mean_squared_error: 1.8606 - loss: 3.4389 - regularization_loss: 0.9851 - total_loss: 4.4241
Epoch 5/50
111/111 [==============================] - 0s 3ms/step - root_mean_squared_error: 1.8596 - loss: 3.4404 - regularization_loss: 0.8588 - total_loss: 4.2992
Epoch 6/50
111/111 [==============================] - 0s 3ms/step - root_mean_squared_error: 1.8608 - loss: 3.4685 - regularization_loss: 0.7493 - total_loss: 4.2177
Epoc

INFO:tensorflow:Assets written to: product_recommender_model\assets


INFO:tensorflow:Assets written to: product_recommender_model\assets


Recommendations:
Product Name: ganesha wooden table clock with metal lantern, Category: home, Subcategory: home decor,home decor, Price: 995.0
Product Name: best teacher personalized table lamp, Category: home, Subcategory: home decor, Price: 645.0
Product Name: bee happy personalized wooden table clock, Category: home, Subcategory: home decor, Price: 425.0
Product Name: traditional vase stand table clock, Category: home, Subcategory: home decor, Price: 650.0
Product Name: mickey love personalized table clock, Category: home, Subcategory: home decor, Price: 675.0


C:\Users\Sayantan\AppData\Local\Temp\ipykernel_9564\3935803115.py:91: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index="products", body=search_body)


In [11]:
import pandas as pd
from elasticsearch import Elasticsearch, helpers
import tensorflow as tf
import tensorflow_recommenders as tfrs
from tensorflow.keras.regularizers import l2

# Load the CSV file
df = pd.read_csv('products-all-data.csv')
df = df[['products_id', 'products_name', 'category', 'base_sub_category', 'products_mrp', 'rating', 'rating_count', 'review_count']]
df['products_id'] = df['products_id'].astype(str)

# Define the ProductModel class
class ProductModel(tfrs.Model):
    def __init__(self, df):
        super().__init__()
        embedding_dimension = 32
        self.product_model = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=df['products_id'].unique(), mask_token=None),
            tf.keras.layers.Embedding(len(df['products_id'].unique()) + 1, embedding_dimension)
        ])
        self.rating_model = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation="relu", kernel_regularizer=l2(0.01)),
            tf.keras.layers.Dense(64, activation="relu", kernel_regularizer=l2(0.01)),
            tf.keras.layers.Dense(1)
        ])
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def call(self, features):
        product_embeddings = self.product_model(features["products_id"])
        rating_predictions = self.rating_model(tf.concat([product_embeddings], axis=1))
        return rating_predictions

    def compute_loss(self, features, training=False):
        labels = features["rating"]
        predictions = self(features)
        return self.task(labels=labels, predictions=predictions)

# Convert dataframe to TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices(dict(df))
dataset = dataset.shuffle(10000).batch(256)

# Train and save the model
model = ProductModel(df)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.01))
model.fit(dataset, epochs=50)
model.save('product_recommender_model')

# Initialize Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

# Index products into Elasticsearch
def index_products(df):
    actions = [
        {
            "_index": "products",
            "_id": row['products_id'],
            "_source": {
                "products_id": row['products_id'],
                "products_name": row['products_name'],
                "category": row['category'],
                "base_sub_category": row['base_sub_category'],
                "products_mrp": row['products_mrp']
            }
        }
        for _, row in df.iterrows()
    ]
    helpers.bulk(es, actions)

# Uncomment the next line if you need to re-index the data
# index_products(df)

# Define the function to get similar products
def get_similar_products(product_name, num_recommendations=5):
    search_body = {
        "query": {
            "match": {
                "products_name": product_name
            }
        },
        "size": 50
    }
    res = es.search(index="products", body=search_body)
    if res['hits']['total']['value'] == 0:
        return []
    candidates = []
    for hit in res['hits']['hits']:
        candidates.append({
            "products_name": hit['_source']['products_name'],
            "category": hit['_source']['category'],
            "base_sub_category": hit['_source']['base_sub_category'],
            "products_mrp": hit['_source']['products_mrp']
        })
    sorted_candidates = sorted(candidates, key=lambda x: x["products_mrp"], reverse=True)
    return sorted_candidates[:num_recommendations]

# Example usage
product_name = 'attractive table ringer'
recommendations = get_similar_products(product_name, num_recommendations=5)
print("Recommendations:")
for rec in recommendations:
    print(f"Product Name: {rec['products_name']}, Category: {rec['category']}, Subcategory: {rec['base_sub_category']}, Price: {rec['products_mrp']}")


C:\Users\Sayantan\AppData\Local\Temp\ipykernel_9564\1156120538.py:8: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('products-all-data.csv')


Epoch 1/50
111/111 [==============================] - 1s 2ms/step - root_mean_squared_error: 1.8920 - loss: 3.5547 - regularization_loss: 1.4965 - total_loss: 5.0512
Epoch 2/50
111/111 [==============================] - 0s 2ms/step - root_mean_squared_error: 1.8549 - loss: 3.4132 - regularization_loss: 1.3023 - total_loss: 4.7155
Epoch 3/50
111/111 [==============================] - 0s 2ms/step - root_mean_squared_error: 1.8576 - loss: 3.4373 - regularization_loss: 1.1334 - total_loss: 4.5707
Epoch 4/50
111/111 [==============================] - 0s 2ms/step - root_mean_squared_error: 1.8567 - loss: 3.4316 - regularization_loss: 0.9871 - total_loss: 4.4187
Epoch 5/50
111/111 [==============================] - 0s 3ms/step - root_mean_squared_error: 1.8608 - loss: 3.4449 - regularization_loss: 0.8603 - total_loss: 4.3053
Epoch 6/50
111/111 [==============================] - 0s 2ms/step - root_mean_squared_error: 1.8593 - loss: 3.4586 - regularization_loss: 0.7504 - total_loss: 4.2090
Epoc

INFO:tensorflow:Assets written to: product_recommender_model\assets


INFO:tensorflow:Assets written to: product_recommender_model\assets


Recommendations:
Product Name: bold statement roses table flowers, Category: fresh, Subcategory: flowers, Price: 10545.0
Product Name: rainbow flushes iridescent table flowers, Category: fresh, Subcategory: flowers, Price: 8345.0
Product Name: round table piece, Category: none, Subcategory: flowers, Price: 7140.0
Product Name: hello reindeers table flowers, Category: fresh, Subcategory: flowers, Price: 6145.0
Product Name: pink frosty christmas table flowers, Category: fresh, Subcategory: flowers, Price: 5945.0


C:\Users\Sayantan\AppData\Local\Temp\ipykernel_9564\1156120538.py:86: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index="products", body=search_body)


In [18]:
import pandas as pd
from elasticsearch import Elasticsearch, helpers
import tensorflow as tf
import tensorflow_recommenders as tfrs
from tensorflow.keras.regularizers import l2

# Load the CSV file
df = pd.read_csv('products-all-data.csv')
df = df[['products_id', 'products_name', 'category', 'sub_category', 'products_mrp', 'rating', 'rating_count', 'review_count']]
df['products_id'] = df['products_id'].astype(str)

# Define the ProductModel class
class ProductModel(tfrs.Model):
    def __init__(self, df):
        super().__init__()
        embedding_dimension = 32
        self.product_model = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=df['products_id'].unique(), mask_token=None),
            tf.keras.layers.Embedding(len(df['products_id'].unique()) + 1, embedding_dimension)
        ])
        self.rating_model = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation="relu", kernel_regularizer=l2(0.01)),
            tf.keras.layers.Dense(64, activation="relu", kernel_regularizer=l2(0.01)),
            tf.keras.layers.Dense(1)
        ])
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def call(self, features):
        product_embeddings = self.product_model(features["products_id"])
        rating_predictions = self.rating_model(tf.concat([product_embeddings], axis=1))
        return rating_predictions

    def compute_loss(self, features, training=False):
        labels = features["rating"]
        predictions = self(features)
        return self.task(labels=labels, predictions=predictions)

    def get_product_embeddings(self, product_ids):
        product_ids = [str(pid) for pid in product_ids]
        product_embeddings = self.product_model(tf.constant(product_ids))
        return product_embeddings

# Convert dataframe to TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices(dict(df))
dataset = dataset.shuffle(10000).batch(256)

# Train and save the model
model = ProductModel(df)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.01))
model.fit(dataset, epochs=100)
model.save('product_recommender_model')

# Initialize Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

# Index products into Elasticsearch
def index_products(df):
    actions = [
        {
            "_index": "products",
            "_id": row['products_id'],
            "_source": {
                "products_id": row['products_id'],
                "products_name": row['products_name'],
                "category": row['category'],
                "sub_category": row['sub_category'],
                "products_mrp": row['products_mrp']
            }
        }
        for _, row in df.iterrows()
    ]
    helpers.bulk(es, actions)

# Uncomment the next line if you need to re-index the data
# index_products(df)

# Define the function to get similar products
def get_similar_products(product_name, num_recommendations=5):
    search_body = {
        "query": {
            "match": {
                "products_name": product_name
            }
        },
        "size": 50
    }
    res = es.search(index="products", body=search_body)
    if res['hits']['total']['value'] == 0:
        return []
    candidates = []
    for hit in res['hits']['hits']:
        candidates.append({
            "products_id": hit['_source']['products_id'],
            "products_name": hit['_source']['products_name'],
            "category": hit['_source']['category'],
            "sub_category": hit['_source']['sub_category'],
            "products_mrp": hit['_source']['products_mrp']
        })
    candidate_ids = [candidate["products_id"] for candidate in candidates]
    product_embeddings = model.get_product_embeddings(candidate_ids)
    scores = model.rating_model(product_embeddings)
    for i, score in enumerate(scores):
        candidates[i]["score"] = score.numpy()[0]
    sorted_candidates = sorted(candidates, key=lambda x: x["score"], reverse=True)
    return sorted_candidates[:num_recommendations]

# Example usage
product_name = 'attractive table ringer'
recommendations = get_similar_products(product_name, num_recommendations=5)
print("Recommendations:")
for rec in recommendations:
    print(f"Product Name: {rec['products_name']}, Category: {rec['category']}, Subcategory: {rec['sub_category']}, Price: {rec['products_mrp']}")

C:\Users\Sayantan\AppData\Local\Temp\ipykernel_9564\3483391777.py:8: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('products-all-data.csv')


Epoch 1/100
111/111 [==============================] - 1s 2ms/step - root_mean_squared_error: 1.8975 - loss: 3.5745 - regularization_loss: 1.4979 - total_loss: 5.0724
Epoch 2/100
111/111 [==============================] - 0s 2ms/step - root_mean_squared_error: 1.8549 - loss: 3.4324 - regularization_loss: 1.3035 - total_loss: 4.7359
Epoch 3/100
111/111 [==============================] - 0s 2ms/step - root_mean_squared_error: 1.8598 - loss: 3.4362 - regularization_loss: 1.1345 - total_loss: 4.5707
Epoch 4/100
111/111 [==============================] - 0s 2ms/step - root_mean_squared_error: 1.8612 - loss: 3.4443 - regularization_loss: 0.9880 - total_loss: 4.4323
Epoch 5/100
111/111 [==============================] - 0s 2ms/step - root_mean_squared_error: 1.8583 - loss: 3.4401 - regularization_loss: 0.8612 - total_loss: 4.3013
Epoch 6/100
111/111 [==============================] - 0s 2ms/step - root_mean_squared_error: 1.8610 - loss: 3.4565 - regularization_loss: 0.7509 - total_loss: 4.207

INFO:tensorflow:Assets written to: product_recommender_model\assets


INFO:tensorflow:Assets written to: product_recommender_model\assets


Recommendations:
Product Name: mickey love personalized table clock, Category: home, Subcategory: home decor, Price: 675.0
Product Name: happy diwali personalized wooden table clock, Category: home, Subcategory: home decor, Price: 425.0
Product Name: traditional vase stand table clock, Category: home, Subcategory: home decor, Price: 650.0
Product Name: xoxo personalized table photo frame, Category: home, Subcategory: home decor, Price: 425.0
Product Name: botanical printed table cover with 6 napkins, Category: home, Subcategory: furnishings, Price: 645.0


C:\Users\Sayantan\AppData\Local\Temp\ipykernel_9564\3483391777.py:91: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index="products", body=search_body)


In [23]:
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
from tensorflow.keras.layers import StringLookup, Embedding, Dense
from tensorflow.keras.regularizers import l2

# Load and preprocess the CSV file
df = pd.read_csv('products-all-data.csv')
df = df[['products_id', 'products_name', 'category', 'base_sub_category', 'products_mrp', 'rating', 'rating_count', 'review_count']]
df['products_id'] = df['products_id'].astype(str)

# Define the ProductModel class
class ProductModel(tfrs.Model):
    def __init__(self, df):
        super().__init__()
        embedding_dimension = 32

        self.product_model = tf.keras.Sequential([
            StringLookup(vocabulary=df['products_id'].unique(), mask_token=None),
            Embedding(len(df['products_id'].unique()) + 1, embedding_dimension)
        ])
        self.rating_model = tf.keras.Sequential([
            Dense(256, activation="relu", kernel_regularizer=l2(0.01)),
            Dense(64, activation="relu", kernel_regularizer=l2(0.01)),
            Dense(1)
        ])
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def call(self, features):
        product_embeddings = self.product_model(features["products_id"])
        rating_predictions = self.rating_model(tf.concat([product_embeddings], axis=1))
        return rating_predictions

    def compute_loss(self, features, training=False):
        labels = features["rating"]
        predictions = self(features)
        return self.task(labels=labels, predictions=predictions)

    def get_product_embeddings(self, product_ids):
        product_ids = [str(pid) for pid in product_ids]
        product_embeddings = self.product_model(tf.constant(product_ids))
        return product_embeddings

# Convert dataframe to TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices(dict(df))
dataset = dataset.shuffle(10000).batch(256)

# Train and save the model
model = ProductModel(df)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.01))
model.fit(dataset, epochs=100)
model.save('product_recommender_model_new')

C:\Users\Sayantan\AppData\Local\Temp\ipykernel_9564\3992754290.py:8: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('products-all-data.csv')


Epoch 1/100
111/111 [==============================] - 1s 2ms/step - root_mean_squared_error: 1.8940 - loss: 3.5655 - regularization_loss: 1.4947 - total_loss: 5.0602
Epoch 2/100
111/111 [==============================] - 0s 2ms/step - root_mean_squared_error: 1.8584 - loss: 3.4324 - regularization_loss: 1.3008 - total_loss: 4.7332
Epoch 3/100
111/111 [==============================] - 0s 2ms/step - root_mean_squared_error: 1.8576 - loss: 3.4151 - regularization_loss: 1.1324 - total_loss: 4.5474
Epoch 4/100
111/111 [==============================] - 0s 2ms/step - root_mean_squared_error: 1.8589 - loss: 3.4619 - regularization_loss: 0.9864 - total_loss: 4.4483
Epoch 5/100
111/111 [==============================] - 0s 2ms/step - root_mean_squared_error: 1.8598 - loss: 3.4608 - regularization_loss: 0.8598 - total_loss: 4.3206
Epoch 6/100
111/111 [==============================] - 0s 2ms/step - root_mean_squared_error: 1.8608 - loss: 3.4596 - regularization_loss: 0.7500 - total_loss: 4.209

INFO:tensorflow:Assets written to: product_recommender_model_new\assets


INFO:tensorflow:Assets written to: product_recommender_model_new\assets
